# Run basic singlepoint calculation

Load the aiida profile:


In [ ]:
# Load profile
from aiida import load_profile
load_profile()

We need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin. 
In this example we are using NaCl structure

In [ ]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:

In [ ]:
from aiida.orm import load_code
code = load_code("janus@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [ ]:
from aiida.orm import Str
inputs = {
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "code": code,
    "arch": model.architecture,
    "precision": Str("float64"),
    "struct": structure,
    "model": model,
    "device": Str("cpu"),
}

We must also choose the calculation to perform:

In [ ]:
from aiida.plugins import CalculationFactory
singlepointCalc = CalculationFactory("mlip.sp")

Then we can create the workgraph and define our input variables

In [ ]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("SinglepointWorkGraph")

sp_calc = wg.add_task(
    singlepointCalc,
    name="sp_calc",
)

In [ ]:
wg.tasks.sp_calc.set(
    {**inputs}
)

We can generate the tasks of the workgraph

In [ ]:
wg

In [ ]:
wg.run()

The graph can be visualised

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)

We can then check the output to ensure we are getting the correct output

In [ ]:
wg.tasks.sp_calc.outputs.xyz_output